In [ ]:

######################### This version is meant to be a clone or Dr.Orlandi's version. Using his psych curve etc. goal is to clone behaviour. Will Implement objects.
######################### And then modify from there 



################################################GONNA SWITCH FROM -1,0,1 to 0,1,2
#gonna work in radians


In [ ]:
#imports
import numpy as np
import sklearn.model_selection as sk
import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow.keras.layers import Input
import tensorflow.keras.backend as kb
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.io as sio
import random


In [ ]:
################### GLOBAL VARIABLES

NUMANGLES = 3 #how many orientations each side can take
MAXPROB = .9


ANGLEMAX = np.pi/2
ANGLEMIN = -np.pi/2

LEFTCHOICEVAL = 0  #Value output when choosing left bar
RIGHTCHOICEVAL = 2 #Value output when choosing right bar

UNITS = 256 # Number of neurons in the dense layer
OUTPUT_SIZE = 3 # Number of output states
INPUT_DIMENSION = 2 #dimension of input. it will be 3 when there is attention included into model. 

STIMULUSDURATION = 1 #length of the stimulus (seconds)
PRESTIMULUSDURATION = 1 #length of the pre stimulus (seconds)

FRAMERATE = 2 #(hz)

TOTALFRAMES = int(FRAMERATE*(STIMULUSDURATION + PRESTIMULUSDURATION)) #total number of frames

LASTFRAME = TOTALFRAMES - 1

PRESTIMULUSFRAME = int(PRESTIMULUSDURATION * FRAMERATE - 1)
STIMULUS_FRAME = PRESTIMULUSFRAME + 1

#################################################

In [ ]:
class timeSeries:
    '''Class whose objects are times series of an instance'''
    #Initialize it with an empty time series
    def __init__(self, series):
        self.tSeries = series

In [ ]:
class anglePair:
    def __init__(self, angle1, angle2):
        self.angle1 = angle1
        self.angle2 = angle2

In [ ]:
class timeSeriesCreationMachine:
    '''This class has objects that are machines to create time series. it will have configuration options and a method that spits out time series'''

    def createInputTimeSeries(self, anglePair): 
        inputTimeSeries = timeSeries(np.zeros((TOTALFRAMES, 2)).astype(np.float32)) #stores the trials time series'
        for frame in range(TOTALFRAMES): 
            if frame <= PRESTIMULUSFRAME: #startwith noise in range [-90 deg, 90 deg] when no stimulus present
                inputTimeSeries.tSeries[frame][0] = 0
                inputTimeSeries.tSeries[frame][1] = 0
            else:
                #use the angles corresponding to stimulus
                inputTimeSeries.tSeries[frame][0] = anglePair[0]
                inputTimeSeries.tSeries[frame][1] = anglePair[1]
        return inputTimeSeries
                    
    def createOutputTimeSeries(self, inputTimeSeries):
        numberTrials = len(inputTimeSeries.tSeries) + 1
        outputTimeSeries = timeSeries(np.full((TOTALFRAMES), np.nan)) #stores the response time series
        firstTrainedChoice = np.nan
        oneChoice = True  #when True one choice is made when stim is shown and that choice remains. when False It makes a decision each frame.

        for frame in range(TOTALFRAMES): 
            if frame < PRESTIMULUSFRAME: 
                outputTimeSeries.tSeries[frame] = np.nan
            elif frame == PRESTIMULUSFRAME:
                outputTimeSeries.tSeries[frame] = 1
            elif frame == STIMULUS_FRAME:
                firstTrainedChoice = self.trainedChoice(inputTimeSeries.tSeries[frame][0], inputTimeSeries.tSeries[frame][1])
                outputTimeSeries.tSeries[frame] = firstTrainedChoice
            elif oneChoice == True:
                outputTimeSeries.tSeries[frame] = firstTrainedChoice
            else:
                #use the angles corresponding to stimulus
                outputTimeSeries.tSeries[frame] = self.trainedChoice(inputTimeSeries.tSeries[frame][0], inputTimeSeries.tSeries[frame][1]) ###ONE CONSTANT CHOICE OR CHOICE CHANGES
        return outputTimeSeries                                                                                                           #Right now choice changes

    def difficulty(self, angle1, angle2):
        sameSign = True
        difference = abs( abs(angle1) - abs(angle2) )
        if abs(angle1 + angle2) < abs(angle1)+abs(angle2): #if they are opposite sign
            sameSign = False    
        beta = 0.06 #slope parameter for steepness of difficulty curve (a function of angle separation)
        alpha = .028 #Value of difficulty at angle difference of max
        offset = np.arctanh( 2*alpha - 1 )/beta #shift the graph so at x = 0 f(x) is alpha
        f = .4 #Parameter for the slope of the mapping of difference to input to the difficulty function
        if not sameSign: #the angles are not the same side so add a bias to the difficulty
            y = 0.6  #parameter affecting how much f is changed when the angles are on same side or not
            f = f - y*f
        x = 1/(f*difference) #scale it so small differences are large value and big differences are small
        dif = 1/2 * ( np.tanh( beta * (x + offset) ) + 1 )
        return dif

    def correctChoice(self, angle1, angle2):
        '''input 2 angles return the smaller angle corresponding to more vertical orientation
        Angle 1 is left angle 2 is right. Returns -1 for Left 1 for right
        if they are equal error is thrown because currently only training on non equal angles
        '''
        global LEFTCHOICEVAL, RIGHTCHOICEVAL
        if angle1 < angle2:
            return LEFTCHOICEVAL        #The left bar is oriented more vertically
        elif angle2 < angle1:
            return RIGHTCHOICEVAL        #The right bar is oriented more vertically
        else:
            return "Error. Not expecting equal angles" #throw error if angles are equal

    def trainedChoice(self, angle1, angle2):
        '''Input two angles returns the choice the mouse makes'''
        global MAXPROB, LEFTCHOICEVAL, RIGHTCHOICEVAL
        pCorrect = self.probablilityCorrect(self.difficulty(angle1, angle2))
        if pCorrect > MAXPROB:
            pCorrect = MAXPROB
        cor = self.correctChoice(angle1,angle2)
        n = np.random.random()
        if n <= pCorrect: #if they are certain
            return cor
        else:
            #if not certain then it is a guess
            g = np.random.rand()
            if g < .5:
                return cor
            else:
                if cor == RIGHTCHOICEVAL:
                    return LEFTCHOICEVAL
                else:
                    return RIGHTCHOICEVAL
            
    def probablilityCorrect(self, difficulty):
        p = -1*difficulty + 1
        return p


In [ ]:
class ensembleCreator:
    '''uses the timeSeriesCreator class to create an ensemble of iteration time series'''
    #Generate training data
    def generateEnsemble(self):
        inputList = np.array([]) #list of all trials input time series
        outputList = np.array([])#list of all trials outpu time series
        angleList = np.linspace(ANGLEMAX, ANGLEMIN, NUMANGLES)
        #list that holds the pairs [left angle, right angle]
        anglePairs = np.array([])
        #generate all pairs of angles
        for i in angleList:
            for j in angleList:      #round to deal with floating point error
                decimals = 1
                if (round(i,decimals) != round(j,decimals)) & (-round(i,decimals) != round(j,decimals)): #we dont want any pairs that have equal verticallity (same abs(angle))
                    #add noise to the angles
                    noiseScale = 10 #scales how noise the angles are
                    i += np.random.randn()/noiseScale
                    j += np.random.randn()/noiseScale
                    anglePairs = np.append(anglePairs,[i,j])
        
        anglePairs = anglePairs.reshape(-1,2)
        
        np.random.shuffle(anglePairs)

        numberTrials = len(anglePairs)

        timeSeriesCreator = timeSeriesCreationMachine()


        for trial in range(numberTrials):
            inputTimeSer = timeSeriesCreator.createInputTimeSeries(anglePairs[trial]) #input time series
            outputTimeSer = timeSeriesCreator.createOutputTimeSeries(inputTimeSer) # output time series
            inputList = np.append(inputList, inputTimeSer.tSeries)
            outputList = np.append(outputList, outputTimeSer.tSeries)

        inputList = inputList.reshape(numberTrials,TOTALFRAMES, 2)    
        
        outputList = outputList.reshape(numberTrials, TOTALFRAMES)
       
        return inputList, outputList


In [ ]:
class netModel:
    def _custom_loss(self, y_actual,y_pred): 
        '''calculates the loss using the frame immediately before stimulus and last frame of time series
        The frame before stim is to set p left and p right = 0'''
        l1 = kb.sparse_categorical_crossentropy(y_actual[:, PRESTIMULUSFRAME], y_pred[:, PRESTIMULUSFRAME])
        l2 = kb.sparse_categorical_crossentropy(y_actual[:, LASTFRAME], y_pred[:, LASTFRAME])
        custom_loss = l1 + l2
        return custom_loss

    def _custom_cat(self, y_actual,y_pred):
        '''Checks the accuracy of the final predicted output and compares it to the desired output'''
        l2 = tf.keras.metrics.sparse_categorical_accuracy(y_actual[:, LASTFRAME], y_pred[:, LASTFRAME])
        custom_cat = l2
        return custom_cat
    
    def genTestOutput(self):
        ensembleGen = ensembleCreator()

        trialsData, responseData = ensembleGen.generateEnsemble()

        xTrain, xTest, yTrain, yTest = sk.train_test_split(
                                                            trialsData, 
                                                            responseData, 
                                                            test_size=0.25, 
                                                            random_state=None
                                                            )


    

        model = keras.models.Sequential()
        model.add(
                    Input(
                            shape = (
                                        xTrain.shape[1], 
                                        xTrain.shape[2]
                                        )
                            )
                    )
        model.add(layers.SimpleRNN(UNITS, activation = 'relu', return_sequences = True, name = 'rnn'))
        model.add(layers.BatchNormalization(name = 'batch_norm'))
        model.add(layers.Dense(OUTPUT_SIZE, activation = 'softmax', name = 'outputprobs'))

        model.summary()


        batch_size = 660 # Trials per batch
        MAX_EPOCHS = 100 # Total epochs

        metrr = ['accuracy', self._custom_cat]

        model.compile(
                        loss=self._custom_loss,
                        optimizer="adam",
                        metrics=metrr
                        )

        model.fit(
                    xTrain, 
                    yTrain, 
                    epochs = MAX_EPOCHS, 
                    batch_size = batch_size, 
                    verbose = True, 
                    validation_data = (
                                        xTest, 
                                        yTest
                                        )
                    )
        yTestPredict = model.predict(xTest)
        return yTestPredict


In [ ]:
class main:
    def __init__(self):
        mod = netModel()

        yTestPredict = mod.genTestOutput()


        one = []
        two = []
        three = []

        for a in yTestPredict[0]:
            one.append(a[0])
            two.append(a[1])
            three.append(a[2])

        plt.plot(one, c='r')
        plt.plot(two, c='g')
        plt.plot(three,c='b')

        plt.show()

run = main()